In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('../datasets/surrogate_model/CICIDS2017/df_substitute_0311.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/substitute_model/CICIDS2017/df_substitute_0311.csv'

### Select Feature & modify special word/number

In [ ]:
df = df.drop([' Destination Port', ' Fwd Packet Length Max', ' Fwd Packet Length Mean', ' Bwd Packet Length Min', ' Flow IAT Mean', 
              ' Flow IAT Max', ' Flow IAT Min',' Fwd IAT Std', ' Fwd IAT Min', 'Bwd IAT Total', 
              'Fwd PSH Flags' , ' Fwd URG Flags', ' Bwd URG Flags', ' Min Packet Length', ' Packet Length Mean', 
              ' Packet Length Variance', ' RST Flag Count', ' PSH Flag Count', ' URG Flag Count', 
              ' CWE Flag Count', ' ECE Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' CWE Flag Count', ' CWE Flag Count',
              'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 
              'Init_Win_bytes_forward', ' min_seg_size_forward', 'Active Mean', ' Active Min', 'Idle Mean', ' Idle Max', ' Idle Min'], axis=1)
df 

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Std,...,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_backward,act_data_pkt_fwd,Active Std,Active Max,Idle Std,Label,Binary_class,attack_type
0,55516,1,1,43,71,43,0.000000,71,71.000000,0.000000,...,1,71,-1,0,0.000000,0,0.000000e+00,BENIGN,benign,benign
1,150,2,2,46,46,23,0.000000,23,23.000000,0.000000,...,2,46,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
2,116011937,20,16,2075,594,0,335.146296,426,37.125000,111.847441,...,16,594,362,4,44792.284154,197702,1.353116e+06,BENIGN,benign,benign
3,254113,2,4,37,0,0,26.162951,0,0.000000,0.000000,...,4,0,0,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
4,97026,2,2,60,288,30,0.000000,144,144.000000,0.000000,...,2,288,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261180,23712,2,2,82,190,41,0.000000,95,95.000000,0.000000,...,2,190,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
1261181,97890396,6,6,357,11595,0,145.744640,7240,1932.500000,2695.639126,...,6,11595,235,1,0.000000,292,0.000000e+00,DoS Hulk,NaN,DoS
1261182,32,1,1,0,0,0,0.000000,0,0.000000,0.000000,...,1,0,276,0,0.000000,0,0.000000e+00,BENIGN,benign,benign
1261183,11408070,19,21,10829,22666,0,1303.224781,2896,1079.333333,890.312099,...,21,22666,113,5,0.000000,1408037,0.000000e+00,BENIGN,benign,benign


In [ ]:
# Removing whitespaces in column names.
col_names = [col.replace(' ', '') for col in df.columns]
df.columns = col_names
df.head()

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
0,55516,1,1,43,71,43,0.000000,71,71.000,0.000000,...,1,71,-1,0,0.000000,0,0.000000e+00,BENIGN,benign,benign
1,150,2,2,46,46,23,0.000000,23,23.000,0.000000,...,2,46,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
2,116011937,20,16,2075,594,0,335.146296,426,37.125,111.847441,...,16,594,362,4,44792.284154,197702,1.353116e+06,BENIGN,benign,benign
3,254113,2,4,37,0,0,26.162951,0,0.000,0.000000,...,4,0,0,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
4,97026,2,2,60,288,30,0.000000,144,144.000,0.000000,...,2,288,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign


In [ ]:
# Replace weird characters
label_names = df['Label'].unique()

import re
label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]
label_names, len(label_names)

# Replacing 'Label' column values with new readable values.
labels = df['Label'].unique()
for i in range(0,len(label_names)):
    df['Label'] = df['Label'].replace({labels[i] : label_names[i]})
    
df['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'DoS_Hulk', 'DoS_Slowhttptest',
       'FTPPatator', 'DoS_GoldenEye', 'DoS_slowloris', 'SSHPatator',
       'Web_Attack_Brute_Force', 'Web_Attack_XSS', 'Bot', 'Infiltration',
       'Heartbleed', 'Web_Attack_Sql_Injection'], dtype=object)

In [ ]:
df['Label'].value_counts().to_frame().sort_index(ascending=True)

,count
Label,
BENIGN,1048242
Bot,977
DDoS,64008
DoS_GoldenEye,5143
DoS_Hulk,86425
DoS_Slowhttptest,2614
DoS_slowloris,2693
FTPPatator,2967
Heartbleed,6


In [ ]:
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
0,55516,1,1,43,71,43,0.000000,71,71.000000,0.000000,...,1,71,-1,0,0.000000,0,0.000000e+00,BENIGN,benign,benign
1,150,2,2,46,46,23,0.000000,23,23.000000,0.000000,...,2,46,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
2,116011937,20,16,2075,594,0,335.146296,426,37.125000,111.847441,...,16,594,362,4,44792.284154,197702,1.353116e+06,BENIGN,benign,benign
3,254113,2,4,37,0,0,26.162951,0,0.000000,0.000000,...,4,0,0,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
4,97026,2,2,60,288,30,0.000000,144,144.000000,0.000000,...,2,288,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261180,23712,2,2,82,190,41,0.000000,95,95.000000,0.000000,...,2,190,-1,1,0.000000,0,0.000000e+00,BENIGN,benign,benign
1261181,97890396,6,6,357,11595,0,145.744640,7240,1932.500000,2695.639126,...,6,11595,235,1,0.000000,292,0.000000e+00,DoS_Hulk,NaN,DoS
1261182,32,1,1,0,0,0,0.000000,0,0.000000,0.000000,...,1,0,276,0,0.000000,0,0.000000e+00,BENIGN,benign,benign
1261183,11408070,19,21,10829,22666,0,1303.224781,2896,1079.333333,890.312099,...,21,22666,113,5,0.000000,1408037,0.000000e+00,BENIGN,benign,benign


### Sample dataframe by Benign into smaller size

In [ ]:
from sklearn.model_selection import train_test_split

label_Benign_df1, label_Benign_df2 = train_test_split(df[df['Binary_class'] == 'benign'], test_size=0.5)
label_Malicious_df1, label_Malicious_df2 = train_test_split(df[df['Binary_class'] == 'malicious'], test_size=0.5)
df = pd.concat([label_Benign_df2, label_Malicious_df2])
df


,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
330549,180,2,2,82,114,41,0.000000,57,57.0,0.000000,...,2,114,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
1259953,3,2,0,12,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
213471,5695608,9,5,348,3112,0,70.588597,1613,622.4,831.288939,...,5,3112,75,3,0.000000e+00,212816,0.000000,BENIGN,benign,benign
173789,247277,2,2,64,798,32,0.000000,399,399.0,0.000000,...,2,798,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
1170878,13591346,4,4,108,108,27,0.000000,27,27.0,0.000000,...,4,108,-1,3,1.581899e+06,2237147,951555.009656,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241564,67,1,1,0,6,0,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan
631239,7108819,4,0,24,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,3,0.000000e+00,1012,0.000000,DDoS,malicious,DOS
771874,13,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan
978296,72,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan


### Drop NAN values

In [ ]:
# Checking if there are any NULL values in the dataset.
df.isnull().values.any()

True

In [ ]:
df.dropna(inplace=True)
df.isnull().values.any()

False

In [ ]:
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
330549,180,2,2,82,114,41,0.000000,57,57.0,0.000000,...,2,114,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
1259953,3,2,0,12,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
213471,5695608,9,5,348,3112,0,70.588597,1613,622.4,831.288939,...,5,3112,75,3,0.000000e+00,212816,0.000000,BENIGN,benign,benign
173789,247277,2,2,64,798,32,0.000000,399,399.0,0.000000,...,2,798,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
1170878,13591346,4,4,108,108,27,0.000000,27,27.0,0.000000,...,4,108,-1,3,1.581899e+06,2237147,951555.009656,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241564,67,1,1,0,6,0,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan
631239,7108819,4,0,24,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,3,0.000000e+00,1012,0.000000,DDoS,malicious,DOS
771874,13,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan
978296,72,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan


### drop infinite values

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Label,Binary_class,attack_type
330549,180,2,2,82,114,41,0.000000,57,57.0,0.000000,...,2,114,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
1259953,3,2,0,12,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
213471,5695608,9,5,348,3112,0,70.588597,1613,622.4,831.288939,...,5,3112,75,3,0.000000e+00,212816,0.000000,BENIGN,benign,benign
173789,247277,2,2,64,798,32,0.000000,399,399.0,0.000000,...,2,798,-1,1,0.000000e+00,0,0.000000,BENIGN,benign,benign
1170878,13591346,4,4,108,108,27,0.000000,27,27.0,0.000000,...,4,108,-1,3,1.581899e+06,2237147,951555.009656,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241564,67,1,1,0,6,0,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan
631239,7108819,4,0,24,0,6,0.000000,0,0.0,0.000000,...,0,0,-1,3,0.000000e+00,1012,0.000000,DDoS,malicious,DOS
771874,13,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan
978296,72,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,6,0,0,0.000000e+00,0,0.000000,PortScan,malicious,PortScan


### Save the dataframe

In [ ]:
# save to csv
# df.to_csv('datasets/target_model/CICIDS2017/combined_data_0307_1.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sklearn.metrics as m
import xgboost as xgb
from sklearn.model_selection import train_test_split



#Load Data
# df=pd.read_csv('datasets/target_model/CICIDS2017/combined_data_0307_1.csv')

X = df.drop(columns=['Label', 'Binary_class', 'attack_type'], axis=1)
y = df['Binary_class']

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
df ['Binary_class'] = lb.fit_transform(df['Binary_class'])
labels = df['Binary_class']
labels.unique()

array([0, 1])

In [ ]:
features = df.loc[:, df.columns != 'Binary_class']
features = df.drop(columns=['Label', 'attack_type'], axis=1)
df = features
df

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMin,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMean,BwdPacketLengthStd,...,SubflowFwdPackets,SubflowFwdBytes,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_backward,act_data_pkt_fwd,ActiveStd,ActiveMax,IdleStd,Binary_class
330549,180,2,2,82,114,41,0.000000,57,57.0,0.000000,...,2,82,2,114,-1,1,0.000000e+00,0,0.000000,0
1259953,3,2,0,12,0,6,0.000000,0,0.0,0.000000,...,2,12,0,0,-1,1,0.000000e+00,0,0.000000,0
213471,5695608,9,5,348,3112,0,70.588597,1613,622.4,831.288939,...,9,348,5,3112,75,3,0.000000e+00,212816,0.000000,0
173789,247277,2,2,64,798,32,0.000000,399,399.0,0.000000,...,2,64,2,798,-1,1,0.000000e+00,0,0.000000,0
1170878,13591346,4,4,108,108,27,0.000000,27,27.0,0.000000,...,4,108,4,108,-1,3,1.581899e+06,2237147,951555.009656,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241564,67,1,1,0,6,0,0.000000,6,6.0,0.000000,...,1,0,1,6,0,0,0.000000e+00,0,0.000000,1
631239,7108819,4,0,24,0,6,0.000000,0,0.0,0.000000,...,4,24,0,0,-1,3,0.000000e+00,1012,0.000000,1
771874,13,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,2,1,6,0,0,0.000000e+00,0,0.000000,1
978296,72,1,1,2,6,2,0.000000,6,6.0,0.000000,...,1,2,1,6,0,0,0.000000e+00,0,0.000000,1


In [ ]:
# save to csv
df.to_csv('../datasets/substitute_model/CICIDS2017/combined_data_new_0311_1.csv', index=False)